### Response Completeness Evaluator

The Response Completeness evaluator assesses the quality of an agent response by examining how well it aligns with the provided ground truth. The evaluation is based on the following scoring system:

<pre>
Score 1: Fully incomplete: The response misses all necessary and relevant information compared to the ground truth.
Score 2: Barely complete: The response contains only a small percentage of the necessary information.
Score 3: Moderately complete: The response includes about half of the necessary information.
Score 4: Mostly complete: The response contains most of the necessary information, with only minor omissions.
Score 5: Fully complete: The response perfectly matches all necessary and relevant information from the ground truth.
</pre>

The evaluation requires the following inputs:

Response: The response to be evaluated. (string)
Ground Truth: The correct and complete information against which the response is compared. (string)

The evaluator uses these inputs to determine the completeness score, ensuring that the response meaningfully addresses the query while adhering to the provided definitions and data.

### Initialize Completeness Evaluator


In [ ]:
from azure.ai.evaluation import CompletenessEvaluator , AzureOpenAIModelConfiguration
from pprint import pprint

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint="<azure_endpoint>",
    api_key="<api_key>",
    api_version="<api_version>",
    azure_deployment="<azure_deployment>",
)

completeness_evaluator = CompletenessEvaluator(model_config=model_config)

### Samples

#### Evaluating for a ground_truth and prediction

In [3]:
result = completeness_evaluator(
    response="The capital of Japan",
    ground_truth="The capital of Japan is Tokyo."
)
result

{'completeness': 1.0,
 'completeness_result': 'fail',
 'completeness_threshold': 3,
 'completeness_reason': 'The response does not include the essential information that the capital of Japan is Tokyo, making it fully incomplete. It lacks any relevant claims or statements from the ground truth.'}

In [6]:
result = completeness_evaluator(
    response="The capital of Japan is Tokyo.",
    ground_truth="The capital of Japan is Tokyo."
)
result

{'completeness': 5.0,
 'completeness_result': 'pass',
 'completeness_threshold': 3,
 'completeness_reason': 'The response perfectly matches the ground truth without any omissions or errors, indicating that it is fully complete.'}

#### Evaluating for a conversation

In [7]:
conversation = {"messages": [{"role": "system", "createdAt": 1700030000, "content": "You are a subscription support agent. Your role is to help customers with subscription cancellations, renewals, and upgrades."}, {"run_id": "run_20250310_050", "role": "user", "createdAt": 1700030050, "content": [{"type": "text", "text": "Hi, I want to cancel my premium magazine subscription. I've been thinking about switching to a digital-only option. Can you help me with the cancellation and also provide details on the digital plan?"}]}, {"run_id": "run_20250310_050", "role": "assistant", "createdAt": 1700030055, "content": [{"type": "text", "text": "I\u2019m sorry to hear you\u2019re considering cancellation. I\u2019ll first process your cancellation request for the premium subscription and then provide you with details about our digital plan."}]}, {"run_id": "run_20250310_050", "role": "assistant", "createdAt": 1700030060, "content": [{"type": "tool_call", "tool_call": {"id": "tool_call_20250310_050", "type": "function", "function": {"name": "cancel_subscription", "arguments": {"subscription_id": "PREMIUM789"}}}}]}, {"run_id": "run_20250310_050", "role": "tool", "createdAt": 1700030065, "tool_call_id": "tool_call_20250310_050", "content": [{"type": "tool_result", "tool_result": "{ \"status\": \"cancelled\", \"refund\": \"$10.00 credited\" }"}]}, {"run_id": "run_20250310_051", "role": "assistant", "createdAt": 1700030070, "content": [{"type": "text", "text": "Your premium subscription has been cancelled and a refund of $10.00 has been credited. Now, regarding our digital plan, it offers full access to all online content at a reduced rate. Would you like me to send you more detailed information about the digital plan and how to subscribe?"}]}, {"run_id": "run_20250310_052", "role": "user", "createdAt": 1700030080, "content": [{"type": "text", "text": "Yes, please send me all the details including pricing and any special offers."}]}, {"run_id": "run_20250310_052", "role": "assistant", "createdAt": 1700030085, "content": [{"type": "text", "text": "Our digital plan costs $4.99 per month, and if you subscribe within the next 7 days, you'll get an additional 10% discount for the first three months. It includes access to digital issues, exclusive online articles, and interactive content. Would you like to subscribe to the digital plan now?"}]}, {"run_id": "run_20250310_052", "role": "user", "createdAt": 1700030090, "content": [{"type": "text", "text": "I\u2019m interested. Please sign me up for the digital plan."}]}, {"run_id": "run_20250310_052", "role": "assistant", "createdAt": 1700030095, "content": [{"type": "tool_call", "tool_call": {"id": "tool_call_20250310_051", "type": "function", "function": {"name": "subscribe_digital", "arguments": {"account_number": "555", "plan": "digital", "discount": "10%"}}}}]}, {"run_id": "run_20250310_052", "role": "tool", "createdAt": 1700030100, "tool_call_id": "tool_call_20250310_051", "content": [{"type": "tool_result", "tool_result": "{ \"status\": \"subscribed\", \"subscription_id\": \"DIGI123\" }"}]}, {"run_id": "run_20250310_052", "role": "assistant", "createdAt": 1700030105, "content": [{"type": "text", "text": "You are now subscribed to the digital plan. Your subscription ID is DIGI123. Is there anything else I can help you with today?"}]}], "tools": [{"id": "tool_130", "name": "cancel_subscription", "description": "Cancel a subscription given its ID.", "parameters": {"type": "object", "properties": {"subscription_id": {"type": "string", "description": "The subscription ID to cancel."}}}}, {"id": "tool_131", "name": "subscribe_digital", "description": "Subscribe to a digital plan.", "parameters": {"type": "object", "properties": {"account_number": {"type": "string", "description": "The account number of the customer."}, "plan": {"type": "string", "description": "The subscription plan to subscribe to."}, "discount": {"type": "string", "description": "Any discount applied to the subscription."}}}}]}

result = completeness_evaluator(
    conversation=conversation,
)
result

{'completeness': 1.0,
 'completeness_threshold': 3.0,
 'evaluation_per_turn': {'completeness': [1.0, 1.0, 1.0],
  'completeness_result': ['fail', 'fail', 'fail'],
  'completeness_threshold': [3, 3, 3],
  'completeness_reason': ['The response does not contain any relevant information or claims that could be compared to a ground truth, making it fully incomplete.',
   'The response does not contain any relevant information or claims that could be compared to a ground truth, making it fully incomplete.',
   'The response does not contain any relevant information that can be compared to a ground truth, making it fully incomplete.']}}